In [44]:

import os
import sys
import psutil
import pywinauto
import pyautogui
import time
import random
from PIL import ImageGrab
print(sys.version)


3.12.8 (tags/v3.12.8:2dc476b, Dec  3 2024, 19:30:04) [MSC v.1942 64 bit (AMD64)]


In [ ]:
class WechatMomentsLongCapture:
    def __init__(self):
        self.wechat_hwnd=None
        self.wechat_souyisou_hwnd=None
        # 微信顶部区域高度
        self.top_panel_height=340
        self.task_bar_height=72
        w,h=self.get_screen_size_pillow()
        # 内容高度也就是每次翻页滚动高度
        self.scroll_step=h-self.top_panel_height-self.task_bar_height
        self.screenshot_list=[]
        self.final_image=None
        self.wechat_souyisou_hwnd=  self.get_process_hwnd_list('WeChatAppEx.exe')[0]
    # 获取进程pid
    @staticmethod
    def get_process_pids(process_name='WeChat.exe'):
        PIDs= []
        for proc in psutil.process_iter():
            try:
                pinfo = proc.as_dict(attrs=['pid', 'name'])
            except psutil.NoSuchProcess:
                pass
            else:
                if process_name == pinfo['name']:
                    PIDs.append( pinfo['pid'])
        return PIDs
    
    # 获取指定进程窗口列表
    @staticmethod
    def get_process_hwnd_list(process_name='WeChat.exe'):
        pids=WechatMomentsLongCapture.get_process_pids(process_name)
        hwnd_list_res=[]
        for pid in pids:
            try:
                hwnd_list = pywinauto.findwindows.find_windows(process=pid)
            except Exception as e:
                print(e)
            else:
                hwnd_list_res+=hwnd_list
        return hwnd_list_res
    # 窗口置于前
    @staticmethod
    def bring_window_to_front_by_hwnd(hwnd):
        try:
            # 如果找到窗口句柄，选择第一个窗口
            # 使用 pywinauto 控制窗口
            app = pywinauto. Application().connect(handle=hwnd)
            window = app.window(handle=hwnd)
            # 将窗口置于前台
            window.set_focus()
        except Exception as e:
            print(f"Error: {e}")
    
    @staticmethod
    def get_screen_size_pillow():
        screenshot = ImageGrab.grab()
        width, height = screenshot.size
        return width, height
    def focus_souyisou_window(self):
        self.bring_window_to_front_by_hwnd(self.wechat_souyisou_hwnd)
        time.sleep(random.uniform(0.5,1))
    
    def need_continue_scroll(self):
        # 需要两个以上截屏对比进行判断
        if len(self.screenshot_list)<2:
            return True
        if len(self.screenshot_list)>100:
            return False
        return True
        
    def start_capture(self):
        self.focus_souyisou_window()
        app = pywinauto.Application(backend="uia").connect(handle=self.wechat_souyisou_hwnd)
        window = app.window(handle=self.wechat_souyisou_hwnd)
        # print(window.dump_tree())
        w=window.rectangle().width()
        h=window.rectangle().height()
        # target_control =window.child_window(title="微信", control_type="Pane")
        # 计算宽高
        # width = target_control.rectangle().width()  # 或 rect.right - rect.left
        # height = target_control.rectangle().height() # 或 rect.bottom - rect.top

        print(w,h)
        while self.need_continue_scroll():
            screen=pyautogui.screenshot(region=(0,self.top_panel_height,w,self.scroll_step))
            self.screenshot_list.append(screen)
            pyautogui.scroll(-self.scroll_step)
            time.sleep(random.uniform(0.5,1))
            # screen.save('result22.png')
            # return
        for index,screen in enumerate (self.screenshot_list):
            screen.save(f"screenshot{index}.png")

        # return self.screenshot_list
wc= WechatMomentsLongCapture()
wc.start_capture()

2582 1550


In [55]:
def concatenate_images_vertically(image_paths, output_path):
    """
    拼接图片为一张长图（垂直方向）。
    
    :param image_paths: 图片路径列表
    :param output_path: 输出图片路径
    """
    # 打开所有图片
    images = [Image.open(img_path) for img_path in image_paths]
    
    # 获取每张图片的宽和高
    widths, heights = zip(*(img.size for img in images))
    
    # 计算输出图片的宽和高
    total_height = sum(heights)
    max_width = max(widths)
    
    # 创建空白图片用于拼接
    result_img = Image.new("RGB", (max_width, total_height), (255, 255, 255))
    
    # 将图片依次粘贴到空白图片上
    y_offset = 0
    for img in images:
        result_img.paste(img, (0, y_offset))
        y_offset += img.size[1]
    
    # 保存结果
    result_img.save(output_path)
    print(f"拼接完成，保存至 {output_path}")

def get_image_paths(directory, extensions=("jpg", "jpeg", "png", "gif")):
    """
    获取指定目录中所有图片的路径。

    :param directory: 目标目录路径
    :param extensions: 图片文件的扩展名元组（默认支持 jpg、jpeg、png、gif）
    :return: 包含图片路径的列表
    """
    image_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(extensions):
                image_paths.append(os.path.join(root, file))
    return image_paths
iamge_paths=get_image_paths('.')
concatenate_images_vertically(image_paths=iamge_paths, output_path="result.png")

拼接完成，保存至 result.png
